#### this is an experimental script to test an hypothesis.

In [1]:
import supervision as sv

ds = sv.DetectionDataset.from_pascal_voc(
    images_directory_path=f"LLVIP/infrared/train",
    annotations_directory_path=f"LLVIP/Annotations"
)

train_ds, test_ds = ds.split(split_ratio=0.7,
                             random_state=42, shuffle=True)
len(train_ds), len(test_ds)

(8417, 3608)

In [2]:
import os
# Define directories for your YOLO dataset
yolo_dataset_dir = "LLVIP_YOLO_Dataset_Infrared"
train_images_dir = os.path.join(yolo_dataset_dir, "images", "train")
train_labels_dir = os.path.join(yolo_dataset_dir, "labels", "train")
test_images_dir = os.path.join(yolo_dataset_dir, "images", "test")
test_labels_dir = os.path.join(yolo_dataset_dir, "labels", "test")

In [3]:
# Create directories
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(test_images_dir, exist_ok=True)
os.makedirs(test_labels_dir, exist_ok=True)


In [4]:
# Export train_ds
# Call as_yolo directly on the train_ds object
train_ds.as_yolo(
    images_directory_path=train_images_dir,
    annotations_directory_path=train_labels_dir,
    min_image_area_percentage=0.0,
    max_image_area_percentage=1.0,
    # The class_map will be derived from train_ds.classes automatically
)

In [5]:
# Export test_ds
# Call as_yolo directly on the test_ds object
test_ds.as_yolo(
    images_directory_path=test_images_dir,
    annotations_directory_path=test_labels_dir,
    min_image_area_percentage=0.0,
    max_image_area_percentage=1.0,
)

In [6]:
# Create a data.yaml file (crucial for YOLOv8)
# This file tells YOLOv8 where your data is and what classes you have.
data_yaml_path = os.path.join(yolo_dataset_dir, "data.yaml")
with open(data_yaml_path, 'w') as f:
    f.write(f"path: {os.path.abspath(yolo_dataset_dir)}\n")
    f.write("train: images/train\n")
    f.write("val: images/test\n") # YOLOv8 uses 'val' for validation/testing
    f.write(f"nc: {len(train_ds.classes)}\n") # Number of classes
    f.write("names:\n")
    for i, class_name in enumerate(train_ds.classes):
        f.write(f"  {i}: {class_name}\n")

print(f"YOLO dataset prepared at: {yolo_dataset_dir}")
print(f"Data YAML file created at: {data_yaml_path}")

YOLO dataset prepared at: LLVIP_YOLO_Dataset_Infrared
Data YAML file created at: LLVIP_YOLO_Dataset_Infrared\data.yaml


In [7]:
from ultralytics import YOLO

# Load a pre-trained YOLOv8n (nano) model. You can choose other models like 'yolov8s.pt' (small), 'yolov8m.pt' (medium), etc.
# 'n' is the smallest and fastest, good for quick testing.
model = YOLO('yolov8n.pt')

# Train the model

results = model.train(data='LLVIP_YOLO_Dataset/data.yaml', epochs=100, imgsz=640, batch=16, name='LLVIP_YOLOv8_model', freeze=5)

# The training results (weights, logs, etc.) will be saved in a directory like 'runs/detect/LLVIP_YOLOv8_modelX'

New https://pypi.org/project/ultralytics/8.3.159 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.150  Python-3.10.16 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4090, 24563MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=LLVIP_YOLO_Dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=5, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=LLVIP_YOLOv8_model7, nbs=64, nms=False, 

train: Scanning D:\Users\eniang.eniang\Desktop\pedestrian_detection-in-low-light\LLVIP_YOLO_Dataset\labels\train.cache... 8417 images, 3 backgrounds, 0 corrupt: 100%|██████████| 8417/8417 [00:00<?, ?it/s]


val: Fast image access  (ping: 1.72.6 ms, read: 11.25.5 MB/s, size: 104.9 KB)


val: Scanning D:\Users\eniang.eniang\Desktop\pedestrian_detection-in-low-light\LLVIP_YOLO_Dataset\labels\test.cache... 3608 images, 1 backgrounds, 0 corrupt: 100%|██████████| 3608/3608 [00:00<?, ?it/s]


Plotting labels to d:\Users\eniang.eniang\Desktop\pedestrian_detection-in-low-light\runs\detect\LLVIP_YOLOv8_model7\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to d:\Users\eniang.eniang\Desktop\pedestrian_detection-in-low-light\runs\detect\LLVIP_YOLOv8_model7
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    100/100      1.52G      1.006     0.4291      1.015          6        640: 100%|██████████| 527/527 [00:47<00:00, 11.20it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 113/113 [00:21<00:00,  5.22it/s]

                   all       3608      10102      0.951      0.952       0.98      0.678



100 epochs completed in 1.998 hours.
Optimizer stripped from d:\Users\eniang.eniang\Desktop\pedestrian_detection-in-low-light\runs\detect\LLVIP_YOLOv8_model7\weights\last.pt, 6.2MB
Optimizer stripped from d:\Users\eniang.eniang\Desktop\pedestrian_detection-in-low-light\runs\detect\LLVIP_YOLOv8_model7\weights\best.pt, 6.2MB

Validating d:\Users\eniang.eniang\Desktop\pedestrian_detection-in-low-light\runs\detect\LLVIP_YOLOv8_model7\weights\best.pt...
Ultralytics 8.3.150  Python-3.10.16 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4090, 24563MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 113/113 [00:25<00:00,  4.42it/s]


                   all       3608      10102      0.949      0.953      0.981       0.68
Speed: 0.2ms preprocess, 0.9ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to d:\Users\eniang.eniang\Desktop\pedestrian_detection-in-low-light\runs\detect\LLVIP_YOLOv8_model7


In [8]:
from ultralytics import YOLO
# Load the best trained model weights
model_path = 'runs/detect/LLVIP_YOLOv8_model7/weights/best.pt' # Adjust this path
model = YOLO(model_path)

# Evaluate the model on the validation/test set
# 'data' points to your data.yaml
metrics = model.val(data='LLVIP_YOLO_Dataset_Infrared/data.yaml')

# Print metrics
print(f"mAP50-95: {metrics.box.map}")
print(f"mAP50: {metrics.box.map50}")
print(f"mAP75: {metrics.box.map75}")
print(f"Precision: {metrics.box.p}")
print(f"Recall: {metrics.box.r}")

Ultralytics 8.3.150  Python-3.10.16 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4090, 24563MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 19.03.7 MB/s, size: 102.4 KB)


val: Scanning D:\Users\eniang.eniang\Desktop\pedestrian_detection-in-low-light\LLVIP_YOLO_Dataset_Infrared\labels\test.cache... 3608 images, 1 backgrounds, 0 corrupt: 100%|██████████| 3608/3608 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 226/226 [00:26<00:00,  8.61it/s]


                   all       3608      10102      0.949      0.952      0.981       0.68
Speed: 0.2ms preprocess, 1.3ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to d:\Users\eniang.eniang\Desktop\pedestrian_detection-in-low-light\runs\detect\val7
mAP50-95: 0.6798306094306716
mAP50: 0.9808756157653725
mAP75: 0.783041527412477
Precision: [    0.94945]
Recall: [    0.95248]
